In [2]:
from flask import Flask, render_template, request, redirect, url_for, send_from_directory, jsonify
from datetime import date, time, datetime
import subprocess
import pandas as pd
import os
import requests
import json
import time
from os import scandir, getcwd, listdir
import folium
import tempfile
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = "./uploads/"
app.title = "Procesamiento de videos IA AMVISA"

d = datetime.now()
tiempo=(d.strftime('%d-%m-%Y  %H:%M:%S'))
tiempo2=(d.strftime('%d-%m-%Y'))
tiempo3=(d.strftime('%m'))

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/index_en')
def index_en():
    return render_template('index_eng.html')

@app.route('/upload', methods=['POST'])
def upload():
    
    archivos = request.files.getlist('archivo')
    archivos_ingles = request.files.getlist('archivo-ingles')

    # Rutas para guardar archivos y resultado del script
    ruta_video = "./video/"

    # Guardar archivos
    rutas_archivos = []
    for archivo in archivos:
        ruta_archivo = f"{ruta_video}{archivo.filename}"
        archivo.save(ruta_archivo)
        rutas_archivos.append(ruta_archivo)
        
    for archivo in archivos_ingles:
        ruta_archivo = f"{ruta_video}{archivo.filename}"
        archivo.save(ruta_archivo)
        rutas_archivos.append(ruta_archivo)

    return render_template('inspecciones.html',archivos=archivos)

@app.route('/volver')
def volver():
    return render_template('index.html')

@app.route('/volver_eng')
def volver_eng():
    return render_template('index_eng.html')

@app.route('/inspecciones', methods=['POST'])
def inspecciones():
    
    archivos = request.files.getlist('archivo[]')
    diametros = request.form.getlist('diametros_introducidos[]')
    
    diametros = [d for d in diametros if d.strip()]
    
    with open('diametros.txt', 'w') as f:
        f.write('\n'.join(diametros))
        
    start_time = time.time()
    
    ruta_codigo = "./code/IA_AMVISA_MASIVO_BONITO.py"
    
    numero_videos=len(listdir("./video/"))
    resultado = subprocess.run(['python', ruta_codigo], capture_output=True, text=True)
    
    with open('resultados.json', 'r') as f:
        resultados = json.load(f)
    
    # Ruta de las imágenes
    ruta_imagenes_raices = "./static/resultados/raices/"
    ruta_imagenes_acometida = "./static/resultados/acometidas/"
    ruta_imagenes_grieta = "./static/resultados/grieta/"
    
    # Obtener rutas de las imágenes
    imagenes_raices = [url_for('static', filename=f'resultados/raices/{imagen}') for imagen in os.listdir(ruta_imagenes_raices)]
    imagenes_acometidas = [url_for('static', filename=f'resultados/acometidas/{imagen}') for imagen in os.listdir(ruta_imagenes_acometida)]
    imagenes_grieta = [url_for('static', filename=f'resultados/grieta/{imagen}') for imagen in os.listdir(ruta_imagenes_grieta)]
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    return render_template('resultados.html',archivos=archivos,diametros=diametros,numero_videos=numero_videos,tiempo=tiempo,resultados=resultados, imagenes_raices=imagenes_raices,imagenes_acometidas=imagenes_acometidas,imagenes_grieta=imagenes_grieta,tiempo_ejecucion=execution_time)

@app.route('/resultados_posibles')
def resultados_posibles():
    
    ruta_imagenes_raices = "./static/resultados/raices_probable/"
    ruta_imagenes_acometida = "./static/resultados/acometida_probable/"
    ruta_imagenes_grieta = "./static/resultados/grieta_probable/"   
    
    imagenes_raices = [url_for('static', filename=f'resultados/raices_probable/{imagen}') for imagen in os.listdir(ruta_imagenes_raices)]
    imagenes_acometidas = [url_for('static', filename=f'resultados/acometida_probable/{imagen}') for imagen in os.listdir(ruta_imagenes_acometida)]
    imagenes_grieta = [url_for('static', filename=f'resultados/grieta_probable/{imagen}') for imagen in os.listdir(ruta_imagenes_grieta)]
    
    return render_template('resultados_posibles.html',imagenes_raices=imagenes_raices,imagenes_acometidas=imagenes_acometidas,imagenes_grieta=imagenes_grieta)

@app.route('/ayuda')
def ayuda():
    return render_template('ayuda.html')

@app.route('/ayuda_en')
def ayuda_en():
    return render_template('ayuda_eng.html')

@app.route('/pdf_report', methods=['POST'])
def pdf_report():

    ruta_final="./"
    # Enviar el archivo PDF generado como una descarga
    return send_from_directory(ruta_final,"informe.pdf", as_attachment=True)

@app.route('/guardar_imagen', methods=['POST'])
def guardar_imagen():
    data = request.json  # Obtiene los datos enviados en formato JSON
    
    imagen = data['imagen']  # Obtiene la URL de la imagen
    estado = data['estado']  # Obtiene el estado (correcto o incorrecto)
    carpeta_destino = "./static/resultados/"  # Define la carpeta de destino

    # Definir las rutas de las carpetas de fotos correctas e incorrectas
    ruta_fotos_correctas = os.path.join(carpeta_destino, 'correctas')
    ruta_fotos_incorrectas = os.path.join(carpeta_destino, 'incorrectas')
    
    # Crear las carpetas si no existen
    if not os.path.exists(ruta_fotos_correctas):
        os.makedirs(ruta_fotos_correctas)
    if not os.path.exists(ruta_fotos_incorrectas):
        os.makedirs(ruta_fotos_incorrectas)

    # Agregar la lógica para guardar la imagen en la carpeta seleccionada
    if estado == 'correcta':
        ruta_destino = os.path.join(ruta_fotos_correctas, os.path.basename(imagen))
        # Aquí puedes guardar la imagen en la carpeta correspondiente
        respuesta = {'mensaje': f'La imagen ha sido marcada como correcta.'}
    elif estado == 'incorrecta':
        ruta_destino = os.path.join(ruta_fotos_incorrectas, os.path.basename(imagen))
        # Aquí puedes guardar la imagen en la carpeta correspondiente
        respuesta = {'mensaje': f'La imagen ha sido marcada como incorrecta.'}
    else:
        respuesta = {'mensaje': 'Estado no válido.'}

    # Guardar la imagen en la carpeta seleccionada
    with open(ruta_destino, 'wb') as f:
        f.write(request.data)

    return jsonify(respuesta)


@app.route('/export_csv', methods=['POST'])
def export_csv():
    
    # Ruta de la carpeta final
    ruta_final = os.path.join(app.config['UPLOAD_FOLDER'], 'informe')

    # Retorna el archivo CSV para su descarga
    return send_from_directory(ruta_final, "FORMATO_INCIDENCIAS.csv", as_attachment=True)

@app.route('/export_excel', methods=['POST'])
def export_excel():
    
    # Ruta de la carpeta final
    ruta_final = os.path.join(app.config['UPLOAD_FOLDER'], 'informe')
    
    read_file = pd.read_csv("./uploads/informe/FORMATO_INCIDENCIAS.csv")
    read_file.to_excel("./uploads/informe/FORMATO_INCIDENCIAS.xlsx", index=None, header=True)

    # Retorna el archivo CSV para su descarga
    return send_from_directory(ruta_final, "FORMATO_INCIDENCIAS.xlsx", as_attachment=True)

@app.route('/export_manual', methods=['GET'])
def export_manual():

    # Ruta de la carpeta final
    ruta_final = "./static/"

    # Retorna el archivo CSV para su descarga
    return send_from_directory(ruta_final, "Manual de Usuario.pdf", as_attachment=True)

@app.route('/export_manual_eng', methods=['GET'])
def export_manual_eng():

    # Ruta de la carpeta final
    ruta_final = "./static/"

    # Retorna el archivo CSV para su descarga
    return send_from_directory(ruta_final, "User Manual.pdf", as_attachment=True)

@app.route('/export_informe', methods=['POST'])
def export_informe():

    # Ruta de la carpeta final
    ruta_final = os.path.join(app.config['UPLOAD_FOLDER'], 'informe')

    # Retorna el archivo CSV para su descarga
    return send_from_directory(ruta_final, "Informe_Videos_Hechos.csv", as_attachment=True)

@app.route('/export_informe_excel', methods=['POST'])
def export_informe_excel():
    
    ruta_final = os.path.join(app.config['UPLOAD_FOLDER'], 'informe')
    
    read_file = pd.read_csv("./uploads/informe/Informe_Videos_Hechos.csv")
    read_file.to_excel("./uploads/informe/Informe_Videos_Hechos.xlsx", index=None, header=True)

    return send_from_directory(ruta_final, "Informe_Videos_Hechos.xlsx", as_attachment=True)

if __name__ == '__main__':
    #from waitress import serve
    #serve(app, host="0.0.0.0", port=8080)
    app.run(host='10.20.16.65', port=8000)

#if __name__ == '__main__':
#    app.run('localhost', 4449)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://10.20.16.65:8000
Press CTRL+C to quit
10.20.16.65 - - [05/Jun/2024 10:37:51] "GET / HTTP/1.1" 200 -
10.20.16.65 - - [05/Jun/2024 10:37:51] "GET /static/Logo_Sacyr_copia.png HTTP/1.1" 304 -
10.20.16.65 - - [05/Jun/2024 10:37:51] "GET /static/images/home.jpg HTTP/1.1" 304 -
10.20.16.65 - - [05/Jun/2024 10:38:33] "POST /upload HTTP/1.1" 200 -
10.20.16.65 - - [05/Jun/2024 10:38:33] "GET /static/Logo_Sacyr_copia.png HTTP/1.1" 304 -
10.20.16.65 - - [05/Jun/2024 10:38:33] "GET /static/images/home.jpg HTTP/1.1" 304 -
10.20.16.65 - - [05/Jun/2024 10:41:27] "POST /inspecciones HTTP/1.1" 200 -
10.20.16.65 - - [05/Jun/2024 10:41:27] "GET /static/Logo_Sacyr_copia.png HTTP/1.1" 304 -
10.20.16.65 - - [05/Jun/2024 10:41:27] "GET /static/resultados/acometidas/721-8953.mp4_0_frame94.jpg HTTP/1.1" 200 -
10.20.16.65 - - [05/Jun/2024 10:41:27] "GET /static/resultados/raices/9391-9390.mp4_0_frame9.jpg HTTP/1.1" 200 -
10.20.16.65 - - [05/Jun/2024 10:41:27] "GET /static/images/home.jpg HT